# Project Objectives:
1. **Analyse Usage and Demand Patterns:** Examine the extensive trip data available in Co-Wheels’ booking system (TripIQ) to identify patterns in vehicle usage and demand across different locations and times.
2. **Design and Develop a Pricing Model and Tool:**
     1. Create a pricing model that incorporates fixed and variable costs, including fuel and electricity, to determine optimal hourly and daily rates for different locations and times.
     2. Develop a straightforward tool that allows Co-Wheels to input various cost factors and receive tailored pricing options based on location, demand, and seasonal variations.   
4. **Evaluate Seasonal and Temporal Variations:** Assess the impact of seasonal changes and time-of-day variations on car-sharing demand and integrate these factors into the pricing model.
5. **Assess Profitability and Utilisation Impact:** Model potential outcomes of different pricing strategies to evaluate their impact on profitability and vehicle utilisation rates in various locations.
6. **Validate pricing tool:** Test the pricing tool with real-world data to ensure its accuracy and effectiveness in optimising Co-Wheels’ pricing strategy.

# 1. Loading libraries and data

In [ ]:
pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.5 MB/s eta 0:00:00


In [ ]:
# Import required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import warnings
from scipy import stats
from matplotlib.ticker import FuncFormatter
from category_encoders import BinaryEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
import joblib

# Display all the columns of the dataframe
pd.pandas.set_option('display.max_columns', None)

# Ignore all warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

# Data location
DATA_PATH = "/content/drive/MyDrive/MSc Dissertation/data/"
# Encoders location
ENCODERS_PATH = "/content/drive/MyDrive/MSc Dissertation/encoders/"
# Model location
MODEL_PATH = "/content/drive/MyDrive/MSc Dissertation/models/"

ValueError: mount failed

## 1.1. Load data

In [ ]:
df_transformed = pd.read_csv(DATA_PATH + 'transformed_dataset.csv')
df = pd.read_csv(DATA_PATH + 'scaled_dataset.csv')

print('Scaled data: ', df.shape)
print('Transformed data: ', df_transformed.shape)

In [ ]:
df.head()

In [ ]:
df_transformed.head()

In [ ]:
# Convert to pandas datetime object
df['booking_billed_start'] = pd.to_datetime(df['booking_billed_start'])
df_transformed['booking_billed_start'] = pd.to_datetime(df_transformed['booking_billed_start'])

# Sort data by booking_actual_start to ensure temporal order
df = df.sort_values(by='booking_billed_start').reset_index(drop=True)
df_transformed = df_transformed.sort_values(by='booking_billed_start').reset_index(drop=True)

# Drop unnecessary columns
df.drop(columns=['vehicle_description', 'vehicle_registration', 'booking_actual_start',
                 'booking_actual_end', 'booking_billed_end', 'booking_created_at'], inplace=True)

df.shape

In [ ]:
df.head()

In [ ]:
df_transformed.head()

## 1.2 Load Encoders

In [ ]:
# Load the encoders
binary_encoder = joblib.load(ENCODERS_PATH + 'binary_encoder.pkl')
one_hot_encoder = joblib.load(ENCODERS_PATH + 'one_hot_encoder.pkl')

# 2. Model Developing Code

In [ ]:
# Define function to create and compile neural network model
def create_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
# Function to train and evaluate models
def train_evaluate_models(df):
    vehicle_types = [col for col in df.columns if col.startswith('Vehicle Type_')]
    results = {}
    predictions_df = pd.DataFrame()

    for vehicle_type in vehicle_types:
        vehicle_data = df[df[vehicle_type] == 1].copy()

        # Sort the data based on time
        vehicle_data = vehicle_data.sort_values(by='booking_billed_start')

        # Define features and target for the current vehicle type
        features_vehicle = vehicle_data.drop(columns=['hourly_rate', 'daily_rate', 'booking_billed_start', 'booking_id'])

        target_hourly_vehicle = vehicle_data['hourly_rate']
        target_daily_vehicle = vehicle_data['daily_rate']

        # Split the data based on time
        split_ratio = 0.8
        split_index_vehicle = int(len(vehicle_data) * split_ratio)

        X_train = features_vehicle.iloc[:split_index_vehicle]
        X_test = features_vehicle.iloc[split_index_vehicle:]
        X_test_copy = vehicle_data.drop(columns=['hourly_rate', 'daily_rate']).iloc[split_index_vehicle:]
        y_train_hourly = target_hourly_vehicle.iloc[:split_index_vehicle]
        y_test_hourly = target_hourly_vehicle.iloc[split_index_vehicle:]
        y_train_daily = target_daily_vehicle.iloc[:split_index_vehicle]
        y_test_daily = target_daily_vehicle.iloc[split_index_vehicle:]

        # Set values to None
        hourly_mae = None
        hourly_rmse = None
        daily_mae = None
        daily_rmse = None
        y_pred_hourly = None
        y_pred_daily = None


        if 'City' in vehicle_type or '7 Seater' in vehicle_type:
            # Normalize the features
            scaler = MinMaxScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            # Save the scaler
            joblib.dump(scaler, ENCODERS_PATH + f'scaler_{vehicle_type}.pkl')

            # Neural Network for both hourly and daily rates
            model_hourly = create_nn_model(X_train.shape[1])
            model_hourly.fit(X_train, y_train_hourly, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
            model_hourly.save(MODEL_PATH + f'{vehicle_type.replace("Vehicle Type_", "")}_nn_hourly_rate_model.keras')

            # Evaluate the hourly rate model
            y_pred_hourly = model_hourly.predict(X_test)
            hourly_mae = np.round(mean_absolute_error(y_test_hourly, y_pred_hourly), 5)
            hourly_rmse = np.round(np.sqrt(mean_squared_error(y_test_hourly, y_pred_hourly)), 5)

            model_daily = create_nn_model(X_train.shape[1])
            model_daily.fit(X_train, y_train_daily, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
            model_daily.save(MODEL_PATH + f'{vehicle_type.replace("Vehicle Type_", "")}_nn_daily_rate_model.keras')

            # Evaluate the daily rate model
            y_pred_daily = model_daily.predict(X_test)
            daily_mae = np.round(mean_absolute_error(y_test_daily, y_pred_daily), 5)
            daily_rmse = np.round(np.sqrt(mean_squared_error(y_test_daily, y_pred_daily)), 5)

            print(f"Vehicle Type: {vehicle_type.replace('Vehicle Type_', '')}")
            print("Hourly Rate Model - MAE:", hourly_mae)
            print("Hourly Rate Model - RMSE:", hourly_rmse)
            print("Daily Rate Model - MAE:", daily_mae)
            print("Daily Rate Model - RMSE:", daily_rmse)
            print("\n")

        elif 'Everyday' in vehicle_type or 'Van' in vehicle_type:
            # XGBRegressor for both hourly and daily rates
            model_hourly = XGBRegressor()
            model_hourly.fit(X_train, y_train_hourly)
            joblib.dump(model_hourly, MODEL_PATH + f'{vehicle_type.replace("Vehicle Type_", "")}_xgb_hourly_rate_model.pkl')

            model_daily = XGBRegressor()
            model_daily.fit(X_train, y_train_daily)
            joblib.dump(model_daily, MODEL_PATH + f'{vehicle_type.replace("Vehicle Type_", "")}_xgb_daily_rate_model.pkl')

            # Evaluate models
            y_pred_hourly = model_hourly.predict(X_test)
            y_pred_daily = model_daily.predict(X_test)

            # hourly and daily mae and rmse
            hourly_mae = np.round(mean_absolute_error(y_test_hourly, y_pred_hourly), 5)
            hourly_rmse = np.round(np.sqrt(mean_squared_error(y_test_hourly, y_pred_hourly)), 5)
            daily_mae = np.round(mean_absolute_error(y_test_daily, y_pred_daily), 5)
            daily_rmse = np.round(np.sqrt(mean_squared_error(y_test_daily, y_pred_daily)), 5)

            print(f"Vehicle Type: {vehicle_type.replace('Vehicle Type_', '')}")
            print("Hourly Rate Model - MAE:", hourly_mae)
            print("Hourly Rate Model - RMSE:", hourly_rmse)
            print("Daily Rate Model - MAE:", daily_mae)
            print("Daily Rate Model - RMSE:", daily_rmse)
            print("\n")

        elif 'Family' in vehicle_type:
            # XGBRegressor for hourly rate, DecisionTreeRegressor for daily rate
            model_hourly = XGBRegressor()
            model_hourly.fit(X_train, y_train_hourly)
            joblib.dump(model_hourly, MODEL_PATH + f'{vehicle_type.replace("Vehicle Type_", "")}_xgb_hourly_rate_model.pkl')

            model_daily = DecisionTreeRegressor()
            model_daily.fit(X_train, y_train_daily)
            joblib.dump(model_daily, MODEL_PATH + f'{vehicle_type.replace("Vehicle Type_", "")}_dt_daily_rate_model.pkl')

            # Evaluate models
            y_pred_hourly = model_hourly.predict(X_test)
            y_pred_daily = model_daily.predict(X_test)

            # hourly and daily mae and rmse
            hourly_mae = np.round(mean_absolute_error(y_test_hourly, y_pred_hourly), 5)
            hourly_rmse = np.round(np.sqrt(mean_squared_error(y_test_hourly, y_pred_hourly)), 5)
            daily_mae = np.round(mean_absolute_error(y_test_daily, y_pred_daily), 5)
            daily_rmse = np.round(np.sqrt(mean_squared_error(y_test_daily, y_pred_daily)), 5)

            print(f"Vehicle Type: {vehicle_type.replace('Vehicle Type_', '')}")
            print("Hourly Rate Model - MAE:", hourly_mae)
            print("Hourly Rate Model - RMSE:", hourly_rmse)
            print("Daily Rate Model - MAE:", daily_mae)
            print("Daily Rate Model - RMSE:", daily_rmse)
            print("\n")

        # BinaryEncoder inverse transform the location columns
        location_columns = [col for col in X_test_copy.columns if col.startswith('location_')]
        X_test_copy['location'] = binary_encoder.inverse_transform(X_test_copy[location_columns])['location']

        # Add predictions to the dataframe
        temp_df = pd.DataFrame({
            'vehicle_type': vehicle_type.replace('Vehicle Type_', ''),
            'location': X_test_copy['location'],
            'booking_billed_start': X_test_copy['booking_billed_start'],
            'booking_created_at_hour': X_test_copy['booking_created_at_hour'],
            'booking_id': X_test_copy['booking_id'],
            'booking_mileage': X_test_copy['booking_mileage'],
            'booking_rates_hours': X_test_copy['booking_rates_hours'],
            'booking_rates_24hours': X_test_copy['booking_rates_24hours'],
            'per_mile': X_test_copy['per_mile'],
            'booking_actual_cost_distance': X_test_copy['booking_actual_cost_distance'],
            'booking_actual_cost_time': X_test_copy['booking_actual_cost_time'],
            'booking_actual_cost_total': X_test_copy['booking_actual_cost_total'],
            'actual_hourly': y_test_hourly.tolist(),
            'predicted_hourly': y_pred_hourly.flatten().tolist(),
            'actual_daily': y_test_daily.tolist(),
            'predicted_daily': y_pred_daily.flatten().tolist()
        })

        predictions_df = pd.concat([predictions_df, temp_df], axis=0, ignore_index=True)

        # hourly and daily mae and rmse
        results[vehicle_type.replace('Vehicle Type_', '')] = {
            'hourly_rate': {
                'MAE': hourly_mae,
                'RMSE': hourly_rmse
            },
            'daily_rate': {
                'MAE': daily_mae,
                'RMSE': daily_rmse
            }
        }

    return results, predictions_df

In [ ]:
results, predictions_df = train_evaluate_models(df)

In [ ]:
print(results)

In [ ]:
predictions_df.head()

# 3. Demand Factor

## 3.1. Popular Location Demand Factor

In [ ]:
# define a function for popular location based demand factor
def popular_location_demand_factor(historical_data, location):
    # Calculate the number of bookings for each location
    demand_location = historical_data.groupby('location').size()
    demand_location = pd.DataFrame(demand_location, columns=['bookings'])

    if location not in demand_location.index:
        return 0

    # Identify demanded locations
    threshold = np.int32(np.round(np.percentile(demand_location['bookings'], 75)))  # Example threshold for peak hours
    demand_locations = demand_location[demand_location['bookings'] >= threshold].index.tolist()
    max_count = demand_location['bookings'].max()
    location_demand = np.round((demand_location['bookings'] / max_count).mean(), 2)

    # If location is in demand_locations then location has higher demand than other locations.
    if location in demand_locations:
        return location_demand
    else:
        return 0

## 3.2. Peak Hour Demand Factor

In [ ]:
# define a function for peak hour demand factor
def peak_hour_demand_factor(historical_data, location, hour):
    # select subset of data based on location
    location_data = historical_data[historical_data['location'] == location]

    if location_data.empty:
        return 0

    # seperate hourly bookings with location
    # Aggregate bookings by hour
    hourly_bookings = location_data.groupby('booking_created_at_hour').size()
    hourly_bookings = pd.DataFrame(hourly_bookings, columns=['bookings'])

    # Identify peak hours
    threshold = np.int32(np.round(np.percentile(hourly_bookings['bookings'], 75)))  # Example threshold for peak hours
    peak_hours = hourly_bookings[hourly_bookings['bookings'] >= threshold].index.tolist()
    max_count = hourly_bookings['bookings'].max()
    peak_hour_demand = np.round((hourly_bookings['bookings'] / max_count).mean(), 2)

    # If hour is in peak_hours then it has higher demand than other hours.
    if hour in peak_hours:
        return peak_hour_demand
    else:
        return 0

## 3.3. Overall Demand Factor

In [ ]:
# define a function for demand factor
def demand_factor(historical_data, location, hour):
    hour_demand_factor = peak_hour_demand_factor(historical_data, location, hour)
    location_demand_factor = popular_location_demand_factor(historical_data, location)
    final_demand_factor = np.round(np.mean([hour_demand_factor, location_demand_factor]), 5)
    return final_demand_factor

In [ ]:
# Sort dataset based on booking_billed_start
predictions_df.sort_values(by='booking_billed_start', inplace=True)

In [ ]:
# Apply demand factor to predicted dataframe
predictions_df['demand_factor'] = predictions_df.apply(lambda row: demand_factor(df_transformed, row['location'], row['booking_created_at_hour']), axis=1)

# Use dynamic pricing formula for hourly_rate and daily_rate
predictions_df['adjusted_hourly'] = predictions_df['predicted_hourly'] + (predictions_df['demand_factor'] * predictions_df['predicted_hourly'])
predictions_df['adjusted_daily'] = predictions_df['predicted_daily'] + (predictions_df['demand_factor'] * predictions_df['predicted_daily'])

In [ ]:
predictions_df.head()

# 4. Compare Revenues

## 4.1. Inverse Log Transformation

In [ ]:
numerical_features = ['booking_mileage', 'booking_actual_cost_distance',
                      'booking_actual_cost_time', 'booking_actual_cost_total']

# Apply inverse log transformation to numerical features
for feature in numerical_features:
    predictions_df[feature] = np.expm1(predictions_df[feature])

In [ ]:
predictions_df.head()

## 4.2. Actual Revenue

In [ ]:
# Calculate revenue for the actual values
predictions_df['actual_revenue'] = predictions_df['booking_actual_cost_total']

## 4.3. Dynamically Adjusted Revenue

In [ ]:
# Calculate booking adjusted cost distance
predictions_df['booking_adjusted_cost_distance'] = predictions_df['per_mile'] * predictions_df['booking_mileage']

# Calculate booking adjusted cost time
predictions_df['booking_adjusted_cost_time'] = (predictions_df['booking_rates_hours'] * predictions_df['adjusted_hourly']) + (predictions_df['booking_rates_24hours'] * predictions_df['adjusted_daily'])

In [ ]:
# Calculate revenue for the adjusted values
predictions_df['adjusted_revenue'] = predictions_df['booking_adjusted_cost_distance'] + predictions_df['booking_adjusted_cost_time']

In [ ]:
predictions_df.head()

## 4.4. Plot Comparison between revenues

### 4.4.1. Comparison by location

In [ ]:
# Function to format the y-axis labels
def format_y_axis(value, tick_number):
    return f'{value:,.0f}'

In [ ]:
# Plot actual vs adjusted revenue
def plot_revenue_comparison(df):
    grouped_df = df.groupby(['location'])[['actual_revenue', 'adjusted_revenue']].sum().reset_index()

    # Plotting the data
    plt.figure(figsize=(18, 7))

    locations = grouped_df['location']
    actual_revenue = grouped_df['actual_revenue']
    adjusted_revenue = grouped_df['adjusted_revenue']

    bar_width = 0.35
    index = range(len(locations))

    # Creating bars for actual and adjusted revenue
    plt.bar(index, actual_revenue, bar_width, label='Actual Revenue')
    plt.bar([i + bar_width for i in index], adjusted_revenue, bar_width, label='Adjusted Revenue')

    # Adding labels and title
    plt.xlabel('Location')
    plt.ylabel('Revenue')
    plt.title('Comparison of Actual and Adjusted Revenue by Location')
    plt.xticks([i + bar_width / 2 for i in index], locations)
    plt.legend()

    # Formatting the y-axis
    plt.gca().yaxis.set_major_formatter(FuncFormatter(format_y_axis))
    plt.xticks(rotation=90)

    # Display the plot
    plt.tight_layout()
    plt.show()

In [ ]:
plot_revenue_comparison(predictions_df)

### 4.4.2. Overall Comparison

In [ ]:
# Plot actual vs adjusted revenue
def plot_revenue_comparison_overall(df):
    # sum of actual and adjusted revenues
    actual_adjusted_revenues = predictions_df[['actual_revenue', 'adjusted_revenue']].sum()

    # Plotting the data
    plt.figure(figsize=(15, 6))

    bars = plt.bar(['Actual Revenue', 'Adjusted Revenue'], actual_adjusted_revenues, color=['blue', 'orange'])

    # Adding labels and title
    plt.xlabel('Type of Revenues')
    plt.ylabel('Revenue')
    plt.title('Comparison of Actual and Adjusted Revenues')
    plt.xticks([0, 1], ['Actual Revenue', 'Adjusted Revenue'])

    # Adding legends
    plt.legend(bars, ['Actual Revenue', 'Adjusted Revenue'])

    # Formatting the y-axis
    plt.gca().yaxis.set_major_formatter(FuncFormatter(format_y_axis))
    plt.xticks(rotation=0)

    # add difference of the actual and adjusted revenue on the top of the plot
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, f'£{yval:,.0f}', ha='center', va='bottom', fontsize=15)

    # plot difference line
    plt.plot([0, 1], [actual_adjusted_revenues['actual_revenue'], actual_adjusted_revenues['adjusted_revenue']], color='red', linestyle='--', linewidth=2, marker='o')

    # plot the difference
    actual_adjusted_revenues['difference'] = actual_adjusted_revenues['adjusted_revenue'] - actual_adjusted_revenues['actual_revenue']
    actual_adjusted_revenues['difference_pct'] = np.round(((actual_adjusted_revenues['adjusted_revenue'] - actual_adjusted_revenues['actual_revenue'])/actual_adjusted_revenues['actual_revenue'])*100, 2)
    plt.text(0.5, actual_adjusted_revenues['actual_revenue'] + (actual_adjusted_revenues['actual_revenue']*0.15), f'£{actual_adjusted_revenues["difference"]:,.0f} ({actual_adjusted_revenues["difference_pct"]} %)', ha='center', va='bottom', color='red', fontsize=17)

    # Display the plot
    plt.tight_layout()
    plt.show()

In [ ]:
plot_revenue_comparison_overall(predictions_df)

---